# Segmentation of Cell Nucleus

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
base_path = "gdrive/My\ Drive/35_Nuclear_segmentation/"
%cd gdrive/My Drive/35_Nuclear_segmentation/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
/content/gdrive/My Drive/35_Nuclear_segmentation


## Importing Libraries

In [0]:
import warnings
warnings.filterwarnings('ignore')

import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import tensorflow as tf

## Loading Train Files

In [0]:
IMAGE_HEIGHT = IMAGE_WIDTH = 128
NUM_CHANNELS = 3
TRAIN_PATH = "./Data/train/"
TEST_PATH = "./Data/validation/"

In [0]:
from tqdm import tqdm_notebook

def load_images(Path, is_train = True):
  images = []
  masks = []
  train_files = next(os.walk(Path))[1]
  for n, filename in tqdm_notebook(enumerate(train_files)):
    if n > 5: break
    image_path = Path + filename + '/images/' + filename + '.png'
    image = cv2.imread(image_path)
    image = cv2.resize(image, (IMAGE_HEIGHT, IMAGE_WIDTH), interpolation = cv2.INTER_AREA)
    images.append(image)
    
    if(is_train):

      combined_mask = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 1))

      mask_path = Path + filename + '/masks/'
      for mask_file in next(os.walk(mask_path))[2]:
        mask = cv2.imread(mask_path + mask_file)
        mask = cv2.resize(mask, (IMAGE_HEIGHT, IMAGE_WIDTH), interpolation = cv2.INTER_AREA)
        combined_mask = np.maximum(combined_mask, mask)
      masks.append(combined_mask)

  if (is_train): return images, masks
  else: return images


## Displaying few Samples

In [0]:
#from matplotlib import pyplot as plt
#plt.figure(figsize=(10, 10))


## Defining IOU Metric

In [5]:
from keras import backend as K

def iou(ground_truth, prediction, smooth=1):
  intersection = K.sum(K.abs(ground_truth * prediction), axis=[1,2,3])
  union = K.sum(ground_truth,[1,2,3])+K.sum(prediction,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

Using TensorFlow backend.


## Defining Our Model

In [0]:
def train_input_fn(batch_size=5):
    dataset = tf.data.Dataset.from_tensor_slices((train_images,train_masks))
    dataset = dataset.cache()
    dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset

In [0]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

In [8]:
inputs = tf.keras.layers.Input((IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255) (inputs)

c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = tf.keras.layers.Dropout(0.1) (c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2)) (c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = tf.keras.layers.Dropout(0.1) (c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2)) (c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = tf.keras.layers.Dropout(0.2) (c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2)) (c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = tf.keras.layers.Dropout(0.2) (c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = tf.keras.layers.Dropout(0.3) (c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = tf.keras.layers.Dropout(0.2) (c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = tf.keras.layers.Dropout(0.2) (c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = tf.keras.layers.Dropout(0.1) (c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = tf.keras.layers.Dropout(0.1) (c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

# Note our output is effectively a mask of 128 x 128 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 128, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 16) 448         lambda[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 128, 128, 16) 0           conv2d[0][0]                     
_____________

### Configuring TPU

In [9]:
try:
 device_name = os.environ['COLAB_TPU_ADDR']
 TPU_ADDRESS = 'grpc://' + device_name
 print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
 print('TPU not found')

Found TPU at: grpc://10.40.128.186:8470


### HYPER_PARAMETERS

In [0]:
EPOCHS = 10
LEARNING_RATE = 0.01

In [11]:
tf.__version__

'1.15.0'

### Compiling the model

In [12]:
#tfCompatibleMod = tf.keras.models.Model(model) # cast to type tensorflow expect

opt = tf.train.AdamOptimizer(LEARNING_RATE)

model.compile(
      optimizer=opt,
      loss='binary_crossentropy',
      metrics=[iou])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
tpu_strategy = tf.contrib.tpu.TPUDistributionStrategy(cluster_resolver)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(model,
                                              strategy = tpu_strategy
                                              )

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.40.128.186:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 7162522085075071676)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9122376571066680037)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184

TypeError: ignored

### Initializing Callbacks

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
%load_ext tensorboard

In [0]:
logdir = "./Logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


model_path = "./Models/nuclei_finder_checkpoint.h5"
checkpointer = ModelCheckpoint(model_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          restore_best_weights = True)


"""results = model.fit(X_train, Y_train, validation_split=0.1,
                    batch_size=16, epochs=10, 
                    callbacks=[earlystop, checkpoint])"""

In [0]:
history = tpu_model.fit(train_input_fn,
                        validation_split = 0.1
                        epochs=EPOCHS
                        callbacks = [checkpointer, earlystopper, tensorboard_callback]
                       )